<a href="https://colab.research.google.com/github/NitinAShelke/NLP-with-NLTK-Thapar-Summer-School/blob/main/Day_3_Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization 
#### By Nitin Arvind Shelke

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
df = pd.read_csv("tennis.csv")
df.head()

In [ ]:
df['article_text'][1]

In [ ]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [ ]:
# use the Glove method for word representation
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
# create vectors for the sentences:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [ ]:
# Finding Similarities to Summarize Text
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
# convert the sim_mat similarity matrix into the graph, the nodes in this graph will represent the sentences and 
#the edges will represent the similarity scores between the sentences:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
#Now, let’s summarize text:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(5):
    print("ARTICLE:")
    print(df['article_text'][i])
    print('\n')
    print("SUMMARY:")
    print(ranked_sentences[i][1])
    print('\n')